In [1]:
# env : pixlake
%load_ext autoreload
%autoreload 2

In [2]:
# make you auto compeletion faster
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False

In [3]:
import os
import sys

def get_workstation_spark_path(where_are_you : str) -> str:
    if where_are_you == 'titan':
        return '/home/data/ryanchao2012/lib'
    elif where_are_you == 'thor':
        return '/opt/spark/versions'
    else:
        raise ValueError("wrong work station name")

spark_path = get_workstation_spark_path('thor')

print('You have pyspark version : ', os.listdir(spark_path))
os.environ['PYSPARK_PYTHON'] = sys.executable
# spark-2.3, spark-2.4
# os.environ['SPARK_HOME'] = os.path.join(spark_path,'spark-2.3')

# use spakr 3.1

os.environ['SPARK_HOME'] = '/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

You have pyspark version :  ['spark-2.3', 'spark-3.1.2-bin-hadoop2.7', 'spark-3.0', 'spark-3.0.1-bin-hadoop2.7', 'spark-2.3.4-bin-hadoop2.7', 'spark-2.4.7-bin-hadoop2.7', 'spark-2.4', 'spark-3.1']


In [4]:
os.environ['SPARK_HOME']

'/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

In [5]:
from os.path import join
import pandas as pd
from pyspark.sql import SparkSession as Session
from pyspark.sql import DataFrame
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W, types as T
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
C = F.col

In [6]:
# Not supported on Maven
# https://mvnrepository.com/artifact/graphframes/graphframes?repo=spark-packages
conf = (Conf()
    .set('spark.sql.sources.partitionOverwriteMode', 'dynamic')
    .set('spark.driver.memory', '10g')
    .set('spark.driver.maxResultSize', '1g')
#     .set('spark.jars.packages','graphframes:graphframes:0.6.0-spark3.1-s_2.11')
   )

In [7]:
spark = (Session
     .builder
     .appName('graph-lab')
     .master('local[10]')
     .config(conf=conf)
     .getOrCreate())

22/05/21 00:41:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
spark

In [9]:
# from graphframes import *

# types of algorithm

* 路徑查找(Path Finding), 漫步 (Walk)
    * 最短跳躍數 / 加權最小的穿越路徑
    * 平均最短路徑
    
* 中心性 (Centrality)
    * 網路中的哪些節點更重要?
    * 快速傳播資訊的能力 / 快速橋接不同群體的能力
    
* 社群檢測(Community Detection)
    * 圖的子結構 (Clustering)
    * 子圖如何吸引 or 排斥其他節點

# Path Finding & Walk

## Beam Search

* 從特定的 `節點` 透過 `邊` 向外擴展，如果 `節點` 的外連邊過多，需要選出 `TopN` 個邊
* when `TopN` = 1 --> Greedy Approach

<img src='../assets/bs_1.png'></img>

https://www.debugger.wiki/article/html/163584216127749


### Use Case

* Node : keyword
* edges : article shared between keywords
* wieght : n articles
* hypothesis : article will collect slightly different intent(keyword)
* use beam search to demonstrate (visulize) how intent spread from given input
    * e.g. 幫寶適 --> 尿布 比較
    *               --> 幫寶適 好市多
    *               --> ...

## prune graph to tree (huristic)

* Node : entity (from wiki)
* edges : A entity belongs to B entity
* use prune graph to tree to demonstrate (visulize) how the taxonomy tree expand from given input
    * e.g. 演員 --> 各國演員 --> 台灣演員 --> 台灣女演員 ...
                        --> 美國演員 --> ...

# Community Detection
## Connected Components

<img src='../assets/cc_1.png'></img>

https://en.wikipedia.org/wiki/Component_(graph_theory)


* 可以視為某種降維
* 找出 Connected Component 的演算法，有時又被稱為 Union Find algo

### Use case

1. Node - Article
2. Edge - Visit By a user-session
3. Hypothesis - each user-sesson is the same intent (意圖)
4. Use Connected Components to `cluster the articles(content)`

In [10]:
g = [
    ('s1',['a1','a2']),
    ('s2',['a2','a3']),
    ('s3',['a2','a4'])
]

# 'a1', 'a2', 'a3', 'a4' is a connected component (The content is similar)

edges_sdf = spark.createDataFrame(g,['session_id','article_id'])
edges_sdf.show()

+----------+----------+
|session_id|article_id|
+----------+----------+
|        s1|  [a1, a2]|
|        s2|  [a2, a3]|
|        s3|  [a2, a4]|
+----------+----------+



In [12]:
def find_connected_component(edges_sdf : DataFrame) -> DataFrame:
    pass

## Strongly Connected Component


<img src='../assets/scc_1.png'></img>

https://zh.wikipedia.org/zh-tw/%E5%BC%BA%E8%BF%9E%E9%80%9A%E5%88%86%E9%87%8F

* 非常適合用於分析群組中的類似行為，偏好
* 僅適用於有向圖中

### Use Case

